# Dallas Animal Shelter

## Hypothesis Testing  

**Null Hypothesis**: cats and dogs have similar outcomes when encountered with similar intake type and condition.

**Alternative Hypothesis**: the intake condition and type of cats affect their outcome less than dogs.


## Load in data

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# description of fields: https://www.dallasopendata.com/City-Services/Animals-Inventory/qgg6-h4bd
# https://gis.dallascityhall.com/documents/StaticMaps/Council/2013_Council_PDFs/2013_CouncilDistrictAllA.pdf

In [2]:
original_df = pd.read_csv('FY2020_Dallas_Animal_Shelter_Data.csv', low_memory=False)

In [3]:
original_df.head()

,Animal Id,Animal Type,Animal Breed,Kennel Number,Kennel Status,Tag Type,Activity Number,Activity Sequence,Source Id,Census Tract,Council District,Intake Type,Intake Subtype,Intake Total,Reason,Staff Id,Intake Date,Intake Time,Due Out,Intake Condition,Hold Request,Outcome Type,Outcome Subtype,Outcome Date,Outcome Time,Receipt Number,Impound Number,Service Request Number,Outcome Condition,Chip Status,Animal Origin,Additional Information,Month,Year
0,A0144701,DOG,HAVANESE,VT 12,IMPOUNDED,NaN,NaN,1,P0098773,6301,4,OWNER SURRENDER,GENERAL,1,PERSNLISSU,CDM,11/8/19,15:48:00,11/14/19,APP SICK,NaN,RETURNED TO OWNER,WALK IN,11/9/19,11:31:00,R19-558731,K19-486742,NaN,APP SICK,SCAN CHIP,OVER THE COUNTER,RETURNED TO OWNER,NOV.2019,FY2020
1,A0442587,DOG,TERRIER MIX,FREEZER,IMPOUNDED,NaN,NaN,1,P0492284,7102,2,OWNER SURRENDER,DEAD ON ARRIVAL,1,OTHRINTAKS,CDM,11/10/19,14:18:00,11/10/19,DEAD,NaN,DEAD ON ARRIVAL,DISPOSAL,11/10/19,0:00:00,NaN,K19-486954,NaN,DEAD,SCAN CHIP,OVER THE COUNTER,NaN,NOV.2019,FY2020
2,A0458972,DOG,CATAHOULA,RECEIVING,UNAVAILABLE,NaN,A19-195601,1,P9991718,4600,1,STRAY,AT LARGE,1,OTHER,MG1718,10/3/19,11:08:00,10/3/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,NaN,RETURNED TO OWNER,FIELD,10/3/19,13:36:00,NaN,K19-482022,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN NO CHIP,SWEEP,NaN,OCT.2019,FY2020
3,A0525642,DOG,GERM SHEPHERD,INJD 001,IMPOUNDED,NaN,A19-196573,1,P0903792,16605,8,OWNER SURRENDER,GENERAL,1,OTHER,RA 1549,10/11/19,9:55:00,10/17/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,EMERGENCY RESCUE,TRANSFER,MEDICAL-CONTAGIOUS,10/15/19,17:35:00,NaN,K19-483073,NaN,TREATABLE REHABILITABLE NON-CONTAGIOUS,SCAN CHIP,SWEEP,TAGGED,OCT.2019,FY2020
4,A0565586,DOG,SILKY TERRIER,LFD 119,UNAVAILABLE,NaN,NaN,1,P0890077,6900,1,STRAY,AT LARGE,1,OTHRINTAKS,JR,11/8/19,11:55:00,11/14/19,APP WNL,RESCU ONLY,RETURNED TO OWNER,WALK IN,11/9/19,12:57:00,R19-558750,K19-486694,NaN,APP WNL,SCAN CHIP,OVER THE COUNTER,RETURNED TO OWNER,NOV.2019,FY2020


In [4]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20247 entries, 0 to 20246
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Animal Id               20247 non-null  object 
 1   Animal Type             20247 non-null  object 
 2   Animal Breed            20247 non-null  object 
 3   Kennel Number           20247 non-null  object 
 4   Kennel Status           20247 non-null  object 
 5   Tag Type                0 non-null      float64
 6   Activity Number         10636 non-null  object 
 7   Activity Sequence       20247 non-null  int64  
 8   Source Id               20247 non-null  object 
 9   Census Tract            18146 non-null  object 
 10  Council District        18146 non-null  object 
 11  Intake Type             20247 non-null  object 
 12  Intake Subtype          20247 non-null  object 
 13  Intake Total            20247 non-null  int64  
 14  Reason                  19189 non-null

## Create dataframe with necessary columns

In [5]:
dallas = original_df[['Animal Type', 'Animal Breed', 'Council District', 'Intake Type', 'Intake Subtype', 
                      'Reason', 'Intake Date', 'Intake Time', 'Due Out', 'Intake Condition', 'Outcome Type', 'Outcome Subtype',
                      'Outcome Date', 'Outcome Time', 'Outcome Condition', 'Animal Origin', 'Month']]

In [6]:
dallas.head()

,Animal Type,Animal Breed,Council District,Intake Type,Intake Subtype,Reason,Intake Date,Intake Time,Due Out,Intake Condition,Outcome Type,Outcome Subtype,Outcome Date,Outcome Time,Outcome Condition,Animal Origin,Month
0,DOG,HAVANESE,4,OWNER SURRENDER,GENERAL,PERSNLISSU,11/8/19,15:48:00,11/14/19,APP SICK,RETURNED TO OWNER,WALK IN,11/9/19,11:31:00,APP SICK,OVER THE COUNTER,NOV.2019
1,DOG,TERRIER MIX,2,OWNER SURRENDER,DEAD ON ARRIVAL,OTHRINTAKS,11/10/19,14:18:00,11/10/19,DEAD,DEAD ON ARRIVAL,DISPOSAL,11/10/19,0:00:00,DEAD,OVER THE COUNTER,NOV.2019
2,DOG,CATAHOULA,1,STRAY,AT LARGE,OTHER,10/3/19,11:08:00,10/3/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,RETURNED TO OWNER,FIELD,10/3/19,13:36:00,TREATABLE REHABILITABLE NON-CONTAGIOUS,SWEEP,OCT.2019
3,DOG,GERM SHEPHERD,8,OWNER SURRENDER,GENERAL,OTHER,10/11/19,9:55:00,10/17/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,TRANSFER,MEDICAL-CONTAGIOUS,10/15/19,17:35:00,TREATABLE REHABILITABLE NON-CONTAGIOUS,SWEEP,OCT.2019
4,DOG,SILKY TERRIER,1,STRAY,AT LARGE,OTHRINTAKS,11/8/19,11:55:00,11/14/19,APP WNL,RETURNED TO OWNER,WALK IN,11/9/19,12:57:00,APP WNL,OVER THE COUNTER,NOV.2019


## Missing values and strange datatypes

In [7]:
dallas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20247 entries, 0 to 20246
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Animal Type        20247 non-null  object
 1   Animal Breed       20247 non-null  object
 2   Council District   18146 non-null  object
 3   Intake Type        20247 non-null  object
 4   Intake Subtype     20247 non-null  object
 5   Reason             19189 non-null  object
 6   Intake Date        20247 non-null  object
 7   Intake Time        20247 non-null  object
 8   Due Out            20247 non-null  object
 9   Intake Condition   20247 non-null  object
 10  Outcome Type       20247 non-null  object
 11  Outcome Subtype    20247 non-null  object
 12  Outcome Date       19928 non-null  object
 13  Outcome Time       20247 non-null  object
 14  Outcome Condition  18976 non-null  object
 15  Animal Origin      19189 non-null  object
 16  Month              20247 non-null  objec

In [37]:
dallas[dallas.isnull().any(axis=1) == True].head(5)

,Animal Type,Animal Breed,Council District,Intake Type,Intake Subtype,Reason,Intake Date,Intake Time,Due Out,Intake Condition,Outcome Type,Outcome Subtype,Outcome Date,Outcome Time,Outcome Condition,Animal Origin,Month,Intake DateTime
43,DOG,JACK RUSS TERR,NaN,STRAY,WEB,NaN,1/18/20,23:25:00,1/24/20,NORMAL,LOST EXP,OTHER,2/24/20,0:00:00,NaN,NaN,JAN.2020,1/18/20 23:25:00
44,DOG,BOXER,6,OWNER SURRENDER,DEAD ON ARRIVAL,OTHER,10/25/19,11:36:00,10/25/19,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,DEAD ON ARRIVAL,DISPOSAL,10/25/19,14:19:00,NaN,OVER THE COUNTER,OCT.2019,10/25/19 11:36:00
105,DOG,DACHSHUND,4,STRAY,AT LARGE,OTHRINTAKS,10/15/19,12:05:00,10/21/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,TRANSFER,GENERAL,10/22/19,9:46:00,NaN,OVER THE COUNTER,OCT.2019,10/15/19 12:05:00
112,DOG,LABRADOR RETR,NaN,OWNER SURRENDER,GENERAL,NOTRIGHTFT,2/15/20,16:58:00,2/15/20,APP WNL,ADOPTION,WALK IN,2/15/20,18:09:00,APP WNL,OVER THE COUNTER,FEB.2020,2/15/20 16:58:00
239,CAT,DOMESTIC SH,NaN,STRAY,WEB,NaN,1/12/20,21:25:00,1/13/20,NORMAL,FOUND EXP,OTHER,2/13/20,0:00:00,NaN,NaN,JAN.2020,1/12/20 21:25:00


In [9]:
dallas[dallas['Animal Type'] == 'CAT']['Council District'].value_counts(dropna=False).sort_index()

0         1
1       341
10      111
11      111
12       51
13      151
14       85
2       206
3       269
4       245
5       171
6      1823
7       205
8       251
9       125
NaN     381
Name: Council District, dtype: int64

In [10]:
dallas['Outcome Condition'].value_counts(dropna=False)

APP WNL                                   12567
TREATABLE REHABILITABLE NON-CONTAGIOUS     2972
NaN                                        1271
APP SICK                                    780
UNKNOWN                                     570
UNDERAGE                                    550
UNHEALTHY UNTREATABLE NON-CONTAGIOUS        351
APP INJ                                     337
CRITICAL                                    299
DEAD                                        178
HEALTHY                                     137
FATAL                                        71
TREATABLE MANAGEABLE NON-CONTAGIOUS          65
TREATABLE REHABILITABLE CONTAGIOUS           37
DECEASED                                     32
UNHEALTHY UNTREATABLE CONTAGIOUS             27
TREATABLE MANAGEABLE CONTAGIOUS               3
Name: Outcome Condition, dtype: int64

## Time and Dates

In [35]:
dallas[['Intake Date', 'Intake Time', 'Due Out', 'Outcome Date', 'Outcome Time']].head()

,Intake Date,Intake Time,Due Out,Outcome Date,Outcome Time
0,11/8/19,15:48:00,11/14/19,11/9/19,11:31:00
1,11/10/19,14:18:00,11/10/19,11/10/19,0:00:00
2,10/3/19,11:08:00,10/3/19,10/3/19,13:36:00
3,10/11/19,9:55:00,10/17/19,10/15/19,17:35:00
4,11/8/19,11:55:00,11/14/19,11/9/19,12:57:00


In [38]:
dallas['Intake DateTime'] = dallas['Intake Date'] + ' ' + dallas['Intake Time']
dallas.head()

/Users/cindywong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Animal Type,Animal Breed,Council District,Intake Type,Intake Subtype,Reason,Intake Date,Intake Time,Due Out,Intake Condition,Outcome Type,Outcome Subtype,Outcome Date,Outcome Time,Outcome Condition,Animal Origin,Month,Intake DateTime
0,DOG,HAVANESE,4,OWNER SURRENDER,GENERAL,PERSNLISSU,11/8/19,15:48:00,11/14/19,APP SICK,RETURNED TO OWNER,WALK IN,11/9/19,11:31:00,APP SICK,OVER THE COUNTER,NOV.2019,11/8/19 15:48:00
1,DOG,TERRIER MIX,2,OWNER SURRENDER,DEAD ON ARRIVAL,OTHRINTAKS,11/10/19,14:18:00,11/10/19,DEAD,DEAD ON ARRIVAL,DISPOSAL,11/10/19,0:00:00,DEAD,OVER THE COUNTER,NOV.2019,11/10/19 14:18:00
2,DOG,CATAHOULA,1,STRAY,AT LARGE,OTHER,10/3/19,11:08:00,10/3/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,RETURNED TO OWNER,FIELD,10/3/19,13:36:00,TREATABLE REHABILITABLE NON-CONTAGIOUS,SWEEP,OCT.2019,10/3/19 11:08:00
3,DOG,GERM SHEPHERD,8,OWNER SURRENDER,GENERAL,OTHER,10/11/19,9:55:00,10/17/19,TREATABLE REHABILITABLE NON-CONTAGIOUS,TRANSFER,MEDICAL-CONTAGIOUS,10/15/19,17:35:00,TREATABLE REHABILITABLE NON-CONTAGIOUS,SWEEP,OCT.2019,10/11/19 9:55:00
4,DOG,SILKY TERRIER,1,STRAY,AT LARGE,OTHRINTAKS,11/8/19,11:55:00,11/14/19,APP WNL,RETURNED TO OWNER,WALK IN,11/9/19,12:57:00,APP WNL,OVER THE COUNTER,NOV.2019,11/8/19 11:55:00


## Column Values

In [33]:
dallas['Animal Type'].value_counts()

DOG          15100
CAT           4527
WILDLIFE       537
BIRD            73
LIVESTOCK       10
Name: Animal Type, dtype: int64

In [39]:
print('Intake Type: ', list(dallas['Intake Type'].unique()))
print('\n')
print('Intake Condition: ', list(dallas['Intake Condition'].unique()))
print('\n')
print('Outcome Type: ', list(dallas['Outcome Type'].unique()))
print('\n')
print('Outcome Condition: ', list(dallas['Outcome Condition'].unique()))
print('\n')
print('Animal Origin: ', list(dallas['Animal Origin'].unique()))

Intake Type:  ['OWNER SURRENDER', 'STRAY', 'FOSTER', 'CONFISCATED', 'TRANSFER', 'WILDLIFE', 'TREATMENT', 'KEEPSAFE', 'DISPOS REQ']


Intake Condition:  ['APP SICK', 'DEAD', 'TREATABLE REHABILITABLE NON-CONTAGIOUS', 'APP WNL', 'UNHEALTHY UNTREATABLE NON-CONTAGIOUS', 'CRITICAL', 'NORMAL', 'APP INJ', 'UNKNOWN', 'UNHEALTHY UNTREATABLE CONTAGIOUS', 'TREATABLE MANAGEABLE NON-CONTAGIOUS', 'TREATABLE MANAGEABLE CONTAGIOUS', 'HEALTHY', 'TREATABLE REHABILITABLE CONTAGIOUS', 'UNDERAGE', 'FATAL', 'DECEASED']


Outcome Type:  ['RETURNED TO OWNER', 'DEAD ON ARRIVAL', 'TRANSFER', 'DIED', 'EUTHANIZED', 'ADOPTION', 'FOSTER', 'LOST EXP', 'WILDLIFE', 'TREATMENT', 'FOUND EXP', 'OTHER', 'MISSING', 'DISPOSAL']


Outcome Condition:  ['APP SICK', 'DEAD', 'TREATABLE REHABILITABLE NON-CONTAGIOUS', 'APP WNL', 'UNHEALTHY UNTREATABLE NON-CONTAGIOUS', 'UNKNOWN', 'HEALTHY', 'CRITICAL', nan, 'APP INJ', 'FATAL', 'UNDERAGE', 'TREATABLE MANAGEABLE NON-CONTAGIOUS', 'TREATABLE MANAGEABLE CONTAGIOUS', 'TREATABLE REHABILITA

## Exploratory Data Analysis (EDA)

## Plots and Graphs

## Map